In [37]:
import pandas as pd
import os
import json
import numpy as np

In [38]:
survey_directory = "/Users/kritiaxh/Documents/PoldrackLab/experiment-factory-analysis/out"
surveys = ['demographics_survey__stanford']

survey_results = {
    'age': [],
    'gender_counts': {
        'Male': 0,
        'Female': 0,
        'Non-binary/Third Gender': 0,
        'Prefer not to answer': 0
    }
}

# Define mappings for gender responses
gender_options = {
    0: 'Male',
    1: 'Female',
    2: 'Non-binary/Third Gender',
    3: 'Prefer not to answer'
}

In [39]:
# Process each subject's surveys
for foldername in os.listdir(survey_directory):
    folder_path = os.path.join(survey_directory, foldername)
    # Check if it is indeed a directory
    if os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            if filename.endswith('.csv'):
                # Check if the filename contains any of the specified surveys
                if any(survey in filename for survey in surveys):
                    subject_id = filename.split('_')[0]  # Extracting subject ID
                    
                    df = pd.read_csv(os.path.join(folder_path, filename))
                    
                    # Extracting the age
                    age_row = df[df['text'].str.contains("How old are you (in years)?", na=False)]
                    if not age_row.empty:
                        age_res = age_row['response'].values[0]
                        survey_results['age'].append(age_res)

                    # Extracting the gender
                    gender_row = df[df['text'].str.contains("What is your gender?", na=False)]
                    if not gender_row.empty:
                        gender_res = gender_row['response'].values[0]
                        gender_res = int(gender_res)
                        if gender_res in gender_options:
                            survey_results['gender_counts'][gender_options[gender_res]] += 1
                        

/var/folders/8s/r02gjh_90hqdkx4nmh9szbsw0000gn/T/ipykernel_84073/936198662.py:15: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  age_row = df[df['text'].str.contains("How old are you (in years)?", na=False)]
/var/folders/8s/r02gjh_90hqdkx4nmh9szbsw0000gn/T/ipykernel_84073/936198662.py:15: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  age_row = df[df['text'].str.contains("How old are you (in years)?", na=False)]
/var/folders/8s/r02gjh_90hqdkx4nmh9szbsw0000gn/T/ipykernel_84073/936198662.py:15: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  age_row = df[df['text'].str.contains("How old are you (in years)?", na=False)]
/var/folders/8s/r02gjh_90hqdkx4nmh9szbsw0000gn/T/ipykernel_84073/936198662.py:15: UserWarning: This pattern is i

In [40]:
# Final calculations for age
if survey_results['age']:
    survey_results['age'] = [int(age) for age in survey_results['age']]
    survey_results['age'].append(20) #S019's age is 20 from redcap. They didn't complete the demographics survey
    average_age = np.mean(survey_results['age'])
    std_age = pd.Series(survey_results['age']).std()
else:
    average_age = None
    std_age = None

# Store results in survey_results
survey_results['average_age'] = average_age
survey_results['std_age'] = std_age

print("Survey Results:")
print("Average Age:", survey_results['average_age'])
print("Standard Deviation of Age:", survey_results['std_age'])
print("Gender Counts:", survey_results['gender_counts']) #S019's gender is female. This wasn't added in the code, but add this in the paper

Survey Results:
Average Age: 26.2
Standard Deviation of Age: 8.14673129354533
Gender Counts: {'Male': 15, 'Female': 22, 'Non-binary/Third Gender': 2, 'Prefer not to answer': 0}
